In [ ]:
import math
import os
import uuid
from pyspark.sql.functions import monotonically_increasing_id
from pyspark.sql import SparkSession


StatementMeta(, 55c4cb8d-a6df-455f-9961-7d57e0114d2f, 10, Finished, Available, Finished)

In [ ]:
# Read transactions from Bronze
df = spark.read.format("csv")\
    .option("header", True)\
    .option("inferSchema", True)\
    .load("abfss://DevDolphins_Assignment@onelake.dfs.fabric.microsoft.com/Bronze_Assignment.Lakehouse/Files/raw_data/transactions.csv")


StatementMeta(, d89984a3-6f6b-4c59-983d-dc9482b320dd, 6, Finished, Available, Finished)

In [ ]:
display(df)

StatementMeta(, d89984a3-6f6b-4c59-983d-dc9482b320dd, 8, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, d0ec129f-7c08-48b1-9f64-00c7607ad19a)

In [5]:
%%sql
CREATE TABLE IF NOT EXISTS chunk_tracker (
    id INT,
    last_chunk_written INT
)
USING DELTA;

INSERT INTO chunk_tracker VALUES (1, 0);


StatementMeta(, db218613-1505-4a9d-b149-95da9b013444, 8, Finished, Available, Finished)

<Spark SQL result set with 0 rows and 0 fields>

<Spark SQL result set with 0 rows and 0 fields>

In [6]:
%%sql
select * from chunk_tracker

StatementMeta(, db218613-1505-4a9d-b149-95da9b013444, 9, Finished, Available, Finished)

<Spark SQL result set with 1 rows and 2 fields>

In [ ]:
from pyspark.sql import Row
from pyspark.sql.window import Window
from pyspark.sql.functions import row_number, monotonically_increasing_id

# ----------------------------------------------
# 1. Create `chunk_tracker` table if not exists
# ----------------------------------------------
if not spark.catalog.tableExists("chunk_tracker"):
    print("🛠 Creating 'chunk_tracker' table...")
    initial_df = spark.createDataFrame([Row(id=1, last_chunk_written=0)])
    initial_df.write.mode("overwrite").saveAsTable("chunk_tracker")
else:
    print(":'chunk_tracker' table already exists.")

# ----------------------------------------------
# 2. Load last written chunk number
# ----------------------------------------------
tracker_df = spark.read.table("chunk_tracker")
last_chunk_written = tracker_df.collect()[0]["last_chunk_written"]
next_chunk_number = last_chunk_written + 1

# ----------------------------------------------
# 3. Load raw transactions CSV from OneLake
# ----------------------------------------------
df = spark.read.format("csv") \
    .option("header", True) \
    .option("inferSchema", True) \
    .load("abfss://DevDolphins_Assignment@onelake.dfs.fabric.microsoft.com/"
          "Bronze_Assignment.Lakehouse/Files/raw_data/transactions.csv")

# ----------------------------------------------
# 4. Assign row numbers for chunking
# ----------------------------------------------
df = df.withColumn("row_id", row_number().over(Window.orderBy(monotonically_increasing_id())))

# ----------------------------------------------
# 5. Select the current chunk
# ----------------------------------------------
chunk_size = 10000
start = (next_chunk_number - 1) * chunk_size + 1
end = start + chunk_size - 1

chunk_df = df.filter((df.row_id >= start) & (df.row_id <= end)).drop("row_id").coalesce(1)

# ----------------------------------------------
# 6. Write chunk to Silver Lakehouse
# ----------------------------------------------
output_path = (f"abfss://DevDolphins_Assignment@onelake.dfs.fabric.microsoft.com/"
               f"Chunks_Silver_Data.Lakehouse/Files/ChunkData/Chunk_Part_{next_chunk_number:02d}")

chunk_df.write.mode("overwrite").option("header", True).csv(output_path)

print(f"Chunk {next_chunk_number} written to: {output_path}")

# ----------------------------------------------
# 7. Update the tracking table
# ----------------------------------------------
updated_df = spark.createDataFrame([Row(id=1, last_chunk_written=next_chunk_number)])

# Use overwriteSchema to safely overwrite the row
updated_df.write \
    .mode("overwrite") \
    .option("overwriteSchema", "true") \
    .saveAsTable("chunk_tracker")

print(f"chunk_tracker updated with last_chunk_written = {next_chunk_number}")


StatementMeta(, ca821b82-da93-456a-a7db-0b417da56c78, 3, Finished, Available, Finished)

✅ 'chunk_tracker' table already exists.


✅ Chunk 3 written to: abfss://DevDolphins_Assignment@onelake.dfs.fabric.microsoft.com/Chunks_Silver_Data.Lakehouse/Files/ChunkData/Chunk_Part_03


🔁 chunk_tracker updated with last_chunk_written = 3


In [7]:
%%sql
select * from Chunks_Silver_Data.chunk_part1

StatementMeta(, 6b45ab0e-9cc8-4eee-8954-597cc47935d0, 8, Finished, Available, Finished)

<Spark SQL result set with 1000 rows and 10 fields>

In [8]:
%%sql
select count(*) from Chunks_Silver_Data.chunk_part1

StatementMeta(, 6b45ab0e-9cc8-4eee-8954-597cc47935d0, 9, Finished, Available, Finished)

<Spark SQL result set with 1 rows and 1 fields>